# init

In [ ]:
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get("HF_TOKEN")  # Fetches the token from Colab Secrets
login(HF_TOKEN)

In [ ]:
from transformers.utils import logging

logging.set_verbosity_error()  # Suppress progress bars and unnecessary logs

# Pipelines with hugging face

In [ ]:
from transformers import pipeline

In [ ]:
summarizer = pipeline(task="summarization", model="cnicu/t5-small-booksum") #model used for summarization

text = """
Artificial Intelligence (AI) is transforming various industries by enabling machines to perform tasks
that typically require human intelligence. These tasks include natural language processing, computer
vision, speech recognition, and decision-making. AI models, particularly deep learning models, are
trained on large datasets to recognize patterns and make predictions. In recent years, advancements in
AI have led to applications such as self-driving cars, medical diagnosis, and chatbots. Despite its
benefits, AI also raises ethical concerns, including bias in decision-making and the impact on employment.
As AI continues to evolve, researchers and policymakers must address these challenges to ensure
responsible AI development.
"""

summary = summarizer(text,
                     max_length=50 #limit response to 50 tokens
                     )

print(summary[0]["summary_text"])

AI models, particularly deep learning models, are trained on large datasets to recognize patterns and make predictions. In recent years, advancements in AI have led to applications such as self-driving cars, medical diagnosis, and chatbots


In [ ]:
generator = pipeline("text-generation", model="gpt2")  # Model used for text generation

# Example customer complaint
customer_complaint = "The product was horrible, and the service was very slow."

# Predefined apology response
apology_response = "Dear client, we sincerely apologize for the inconvenience you reported"

# Create the prompt with structured input, this is how the response starts
prompt = f"""Customer complaint: {customer_complaint}.\nCompany's response to the customer: {apology_response}.  """

# Generate a response using the model
outputs = generator(
    prompt,
    max_length=150,  # Limit response to 150 tokens
    pad_token_id=generator.tokenizer.eos_token_id,  # Use EOS token to avoid padding issues (GPT-2 lacks a default pad token)
    truncation=True  # Truncate input if it's too long
)

# Print the generated text
print(outputs[0]["generated_text"])  # Extract and display the generated response

Customer complaint: The product was horrible, and the service was very slow..
Company's response to the customer: Dear client, we sincerely apologize for the inconvenience you reported.    A large number of people have reported some type of lag in their online experience with our online services. We understand that your situation might arise due to the nature of your account, but we would like to know about the way you handled this issue. The complaint was brought to our attention by our Customer Support team in a variety of ways, including by using in-person calls or sending out text messages. Please refer to our "What are the reasons for your complaint?" section for details. We take very seriously the experience of our customers, and strive to take action


In [ ]:
spanish_text = "La inteligencia artificial esta cambiando el planeta"

translator = pipeline(task="translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")

translations = translator(spanish_text,
                          clean_up_tokenization_spaces=True #Fixes spacing issues
                          )

print(translations[0]["translation_text"])

Artificial intelligence is changing the planet.


# transformers arch

🔍 **What’s Different in Traditional NLP terms vs LLMs?**

1️⃣ Encoding in Traditional NLP = Simple static embeddings (Word2Vec, TF-IDF).

1️⃣.1️⃣ Encoding in LLMs = Context-aware representations (transformers track relationships between words).

2️⃣ Decoding in Traditional NLP = Retrieving stored text or using basic grammar rules.

2️⃣.1️⃣Decoding in LLMs = Generating new text using self-attention and autoregressive prediction.


| **Model Type**      | **Description**                                             | **Use Cases**                                      | **Example Models**               |
|---------------------|------------------------------------------------|-------------------------------------------------|--------------------------------|
| **Encoder**      | Reads and **understands** text, but doesn't generate new text | Classification, Sentiment Analysis, Named Entity Recognition (NER) | **BERT, RoBERTa, DistilBERT, ALBERT** |
| **Decoder**    | **Generates** text based on input (auto-regressive)  | Text Completion, Chatbots, Story Writing | **GPT-2, GPT-3, LLaMA, Falcon** |
| **Encoder-Decoder**  | First understands the input (encoder), then generates **transformed output** (decoder) | Summarization, Translation, Question Answering | **T5, BART, mBART, FLAN-T5** |
